In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
%matplotlib widget

In [3]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 

In [4]:
# Get daily cases by regions
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
df_cases = pd.read_csv(latestdir+'/Kommunalt_DB/07_bekraeftede_tilfaelde_pr_dag_pr_kommune.csv',encoding='latin1',delimiter = ';')

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

In [5]:
# Get daily fully vaccinated by reagions
latestsubdir = list(os.walk(path_vacc))[0][1][-1]
latestdir = path_vacc + latestsubdir
df_vacc = pd.read_csv(latestdir+'/Vaccine_DB/FaerdigVacc_kommune_dag.csv',encoding='latin1',delimiter = ';')

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

df_cases = pd.read_csv(latestdir+'/Kommunalt_DB/07_bekraeftede_tilfaelde_pr_dag_pr_kommune.csv',encoding='latin1',delimiter = ';')
df_cases['Dato'] =  pd.to_datetime(df_cases.Dato,format='%Y-%m-%d')
latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir

#Fix Columnaming problem and format dates
df_vacc = df_vacc.rename(columns={'Faerdigvacc. dato': 'Faerdigvacc_dato'})
df_vacc['Faerdigvacc_dato'] =  pd.to_datetime(df_vacc.Faerdigvacc_dato,format='%Y-%m-%d')



In [6]:

df_kommune_kort = pd.read_csv(latestdir+'/Kommunalt_DB/10_Kommune_kort.csv',
                            encoding='latin1', delimiter = ';')
#kommunenavn = df_kommune_kort["Kommunenavn"].dropna()
kommunenavn = ["København","Århus","Odense"] 

def antal_borgere(kommunenavn):
    return df_kommunekort["Antal borgere"][kommunenavn]

In [7]:
for i in kommunenavn:

    kommune_vac_df = df_vacc.loc[df_vacc["kommunenavn"] == i]
    #firstDate = np.datetime64(kommune_df.loc[kommune_df.index[start],'Dato'])-np.timedelta64(1,'D')
    lastDate = np.datetime64(kommune_vac_df.loc[kommune_vac_df.index[-1],'Faerdigvacc_dato'])
    
    # Find number of citizens in region
    df_kommunekort = pd.read_csv(latestdir+'/Kommunalt_DB/10_Kommune_kort.csv',encoding='latin1',
                                 delimiter = ';')
    df_kommunekort = df_kommunekort.set_index("Kommunenavn")

    # Make row of accumulated vaccinations percentage of pop
    kommune_vac_df["Kumuleret sum vac"] = kommune_vac_df['Antal faerdigvacc.'].cumsum() 
    
    kommune_vac_df['Kumuleret procent faerdigvacc.'] = (kommune_vac_df["Kumuleret sum vac"]/antal_borgere(i))*100
    
    # Make row of accumulated infections percentage of pop
    kommune_cases_df = df_cases.loc[df_cases["Kommunenavn"] == i]
    kommune_cases_df["Kumuleret sum tilfælde"] = kommune_cases_df['Bekræftede tilfælde'].cumsum() 
    
    kommune_cases_df['Kumuleret procent smittede'] = (kommune_cases_df["Kumuleret sum tilfælde"]/antal_borgere(i))*100
    
    kommune_merge_df = pd.merge(left=kommune_vac_df, left_on='Faerdigvacc_dato',
         right=kommune_cases_df, right_on='Dato')
    
    plt.plot(kommune_merge_df["Faerdigvacc_dato"],kommune_merge_df["Kumuleret procent faerdigvacc."],label=i,linewidth=0.5)
    plt.plot(kommune_merge_df["Faerdigvacc_dato"],kommune_merge_df["Kumuleret procent smittede"],label=i,linewidth=0.5)

<ipython-input-7-cc107e8ff4e0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kommune_vac_df["Kumuleret sum vac"] = kommune_vac_df['Antal faerdigvacc.'].cumsum()
<ipython-input-7-cc107e8ff4e0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kommune_vac_df['Kumuleret procent faerdigvacc.'] = (kommune_vac_df["Kumuleret sum vac"]/antal_borgere(i))*100
<ipython-input-7-cc107e8ff4e0>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-7-cc107e8ff4e0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kommune_vac_df["Kumuleret sum vac"] = kommune_vac_df['Antal faerdigvacc.'].cumsum()
<ipython-input-7-cc107e8ff4e0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kommune_vac_df['Kumuleret procent faerdigvacc.'] = (kommune_vac_df["Kumuleret sum vac"]/antal_borgere(i))*100
<ipython-input-7-cc107e8ff4e0>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [12]:
plt.legend()

In [15]:
plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [16]:
kommune_merge_df

,Faerdigvacc_dato,kommunenavn,Antal faerdigvacc.,c_kom_num,Kumuleret sum vac,Kumuleret procent faerdigvacc.,Kommune,Kommunenavn,Dato,Bekræftede tilfælde,Kumuleret sum tilfælde,Kumuleret procent smittede
0,2021-01-12,Odense,1,461,1,0.000486,461,Odense,2021-01-12,35,5944,2.891149
1,2021-01-17,Odense,355,461,356,0.173158,461,Odense,2021-01-17,27,6091,2.962650
2,2021-01-18,Odense,387,461,743,0.361394,461,Odense,2021-01-18,30,6121,2.977241
3,2021-01-19,Odense,342,461,1085,0.527742,461,Odense,2021-01-19,35,6156,2.994265
4,2021-01-20,Odense,236,461,1321,0.642532,461,Odense,2021-01-20,21,6177,3.004480
5,2021-01-21,Odense,7,461,1328,0.645936,461,Odense,2021-01-21,29,6206,3.018585
6,2021-01-22,Odense,277,461,1605,0.780669,461,Odense,2021-01-22,27,6233,3.031718
7,2021-01-23,Odense,4,461,1609,0.782614,461,Odense,2021-01-23,18,6251,3.040473
8,2021-01-24,Odense,2,461,1611,0.783587,461,Odense,2021-01-24,13,6264,3.046796
9,2021-01-25,Odense,224,461,1835,0.892540,461,Odense,2021-01-25,15,6279,3.054092
